# Predict hot flow stress curves qualitively by using Avrami recrystallisation model

by Max Weiner (max.weiner@imf.tu-freiberg.de), based on Luton and Sellars https://doi.org/10.1016/0001-6160(69)90049-2

In [ ]:
# only for Google Colab
!wget -N "https://raw.githubusercontent.com/Institute-of-Metal-Forming/notebooks-ilsenburg/refs/heads/main/requirements.txt"
!pip install -r "requirements.txt"

In [ ]:
from scipy import linalg
import numpy as np
import pandas as pd
import plotly.express as px

## Determining Avrami Parameters from Experimental Data

The simplest Avrami-type approach for modeling recrystallisation behavior and other similar processes is given below.

$X(t) = 1 - \exp(-kt^n)$

$X(t)$ is the recrystallized portion of volume, $t$ is the time and $k$ and $n$ are material as well as environment conditions dependent parameters (temperature, deformation, deformation spead, ...).

$k$ and $n$ have to be determined from experimental data for specific conditions. Here some sample data should be used for illustrating the general approach.
Since the Avrami-equation is asymptotic to 0 and 1, as start and end of recrystallisation commonly 0.01 and 0.99 is chosen.

To determine the parameters the Avrami equation is logarithmized twice.

$\log k + n \log t = \log\log\left(\frac{1}{1-X}\right) $

So the following linear equation system can be build.

$\begin{bmatrix}1 & \log t_0 \\ 1 & \log t_1 \end{bmatrix} \begin{pmatrix}\log k\\n\end{pmatrix} = \begin{pmatrix}\log\log\left(\frac{1}{1-X_0}\right)\\\log\log\left(\frac{1}{1-X_1}\right)\end{pmatrix}$

Now a linear regression can be done for $n$ and $\log k$.

For sample purpose, it is assumed, that the recrystallisation starts at $t_0=1\,\mathrm{s}$ and finishes at $t_1=15\,\mathrm{s}$.

The system is build and solved with the `scipy`-package.

In [ ]:
data_t = np.array([1, 15])
data_X = np.array([0.01, 0.99])

In [ ]:
right_side = np.log(np.log(1 / (1 - data_X)))
print(right_side)

In [ ]:
matrix = np.array([[1, np.log(data_t[0])], [1, np.log(data_t[1])]])
print(matrix)

In [ ]:
solution = linalg.solve(matrix, right_side)
print(solution)

So the searched parameters are:

In [ ]:
print("k =", np.exp(solution[0]), "\nn =", solution[1])

## Predicting the Flow Stress Curve Qualitatively

According to Luton and Sellars (link see above) the hot flow stress curves can be calculated at least qualitively with recrystallisation approach of Avrami.

However, a base flow stress curve of the material without recrystallistaion behaviour is needed. Here a simple Ludvig-approach should be used:

In [ ]:
ludvig_k0 = 100  # base flow stress
ludvig_n = 0.2  # exponent


def ludwik(strain):
    return ludvig_k0 * (strain + 0.01) ** ludvig_n

In [ ]:
work_hardening = pd.DataFrame({"strain": np.linspace(0, 2, 100)})
work_hardening["ludwik"] = ludwik(work_hardening["strain"])
px.line(work_hardening, x="strain", y="ludwik")  # sample plot

The Avrami approach is here defined with constant parameters.

In [ ]:
avrami_k = 0.2  # time constant
avrami_n = 2  # exponent


def avrami(time):
    return 1 - np.exp(-avrami_k * time**avrami_n)

In [ ]:
softening = pd.DataFrame({"time": np.linspace(0, 5, 100)})
softening["avrami"] = avrami(softening["time"])
px.line(softening, x="time", y="avrami")  # sample plot

This function calculates the difference in the Avrami-function in a time step `t_increment` going from time `t`.
`t=0` is defined at the start of recrystallization.

In [ ]:
def avrami_increment(time, increment):
    return avrami(time + increment) - avrami(time)

This class is simply needed as data container for the following calculations. It represents a portion of material with defined equivalent deformation.

In [ ]:
class Portion:
    """Class that represents a portion of material, that has experienced an equal deformation"""

    def __init__(self, portion: float):
        self.start_time = -1
        self.portion = portion
        self.start_portion = portion
        self.strain = 0

    def flow_stress(self):
        """returns flow stress of this portion"""
        return ludwik(self.strain)

    def __repr__(self):
        """string representation"""
        return (
            f"({self.start_time}, {self.portion}, {self.strain}, {self.flow_stress()})"
        )

The following function contains the actual algorithm of recrystallisation calculation.
In every time step the recrystallized amount is calculated with Avrami and defined as new material portion.
Each portion has its own state in equivalent strain. 
Only portions with $\varphi > \varphi_c$ are recrystallizing.
The overall flow stress $k_f$ is calculated by arithmetic mean of the flow stresses of all portions.

$k_f = \sum_i k_{fi} x_i$

were $x_i$ is the volume share of this portion (the property `Portion.portion` in code).

In [ ]:
strain_critical = 0.5  # critical strain of recrystallisation
strain_step = 0.02  # numerical step width of deformation
strain_max = 4  # maximum deformation (end of calculation)


def luton_sellars_solve(rate: float):
    """function that predicts the flow stress curve at a given deformation speed"""
    strain = 0
    t = 0
    time_step = strain_step / rate
    portions = [Portion(1)]  # initial portion: whole material, no strain
    results = []

    while strain < strain_max:
        new_portion = 0.0
        for p in portions:
            if p.strain >= strain_critical:
                if p.start_time < 0:
                    # set start_time if this portion recrystallizes first
                    p.start_time = t
                recrystallized_amount = (
                    avrami_increment(t - p.start_time, time_step) * p.start_portion
                )
                p.portion -= recrystallized_amount
                new_portion += recrystallized_amount
            p.strain += strain_step
        if new_portion > 0:
            portions.append(Portion(new_portion))
        strain += strain_step
        t += time_step
        results.append(
            (t, strain, sum((p.portion * p.flow_stress() for p in portions)))
        )
    return pd.DataFrame(results, columns=["t", "strain", "flow_stress"])

Last, this model is calculated for different $\dot{\varphi}$.

In [ ]:
rates = [0.1, 0.2, 0.5, 1.0]

results = pd.concat(
    [luton_sellars_solve(rate) for rate in rates],
    keys=rates,
).reset_index(level=0, names="rate")
results

In [ ]:
px.line(results, x="strain", y="flow_stress", color="rate")